# Batch Isochrones
Run batch Conveyal queries for multiple origins

Setup requires:
 - A current Conveyal token (e.g. 'bearer 1234abcd...') saved at `config/.auth`. This token can be obtained with [the network tab of your browser's dev tools](https://developers.google.com/web/tools/chrome-devtools/network/), by logging into your Conveyal account, navigating to a project, finding a network request to api.conveyal.com, and inspecting the request Authorization headers.
 - An appropriate analysis request template, copied as JSON from the analysis panel of the Conveyal user interface. Relevant properties include `destinationPointSetIds`, `projectId` (id of the project in Conveyal), `regionId`, and `modificationIds`
 - An array of origin points, as a JSON array with properties "name", "fromLat", and "fromLon" saved to `config/origins.json`.

After the setup cell in this notebook, there are cells to: 
 - Create a .geojson file of isochrones for each origin. Each file has features for default travel time cutoffs (0 minutes to 90 minutes, at 15 minute increments) and percentiles.
 - Create a .csv file of accessibility statistics for each origin. These statistics will based on "cutoffs" at minute increments up to 120 minutes, for the [decay function](https://docs.conveyal.com/learn-more/decay-functions), travel time percentiles, and destination pointsets (opportunity datasets) specified in the analysis request.

## Setup

In [1]:
import csv
import requests
import numpy
import json
import matplotlib.pyplot as plt
import geojsoncontour
import geojson
import rasterio
import shapely_geojson
from shapely.geometry import mapping, shape, MultiPolygon
from struct import unpack

url = 'https://api.conveyal.com/api/analysis'

# Authorization header copied from DevTools Network request
token = open('config/.auth').readline().strip()

# Copy from Advanced Settings in the analysis panel
# Ensure that the scenario, variantNumber, and other parameters are set correctly
profileRequest = json.load(open('config/profile-requests/london-demo-2.json'))

# Load from a JSON array with properties "name", "fromLat", and "fromLon" for each point
origins = json.load(open('config/origins.json'))

isochroneIntervals = range(0, 90, 15)

percentileValues = profileRequest.percentiles

headers = {
    'Authorization': token
}

## Download geotiffs and process isochrones

In [ ]:
headers['Accept'] = 'image/tiff'

# Loop over origins
for origin in origins:
    originName = origin['name']
    profileRequest['fromLat'] = origin['fromLat']
    profileRequest['fromLon'] = origin['fromLon']
    
    print('Processing ' + originName)
    
    # Request a tiff from Conveyal Analysis
    r = requests.post(url, headers = headers, json = profileRequest, verify = False)
    
    if r.status_code == 202:
        print('Routing engine starting, try again in a few minutes.')
    
    elif r.status_code == 403:
        print('Unauthorized access. Your authorization token may be invalid or expired.')
        
    elif r.status_code != 200:
        print('Error: ' + r.text)
        
    else:
        # Save response from Conveyal Analysis to a local .tiff file
        with open(originName + '.tiff', 'wb') as f:
            for chunk in r.iter_content(chunk_size=128):
                f.write(chunk)
                
        f.close()
                
        tiff = rasterio.open(originName + '.tiff')
        print(tiff)
        crsEpsg = tiff.crs.to_string()

        # Set up tiff processing        
        latRange = numpy.arange(tiff.bounds.top, tiff.bounds.bottom, -tiff.res[1])
        lonRange = numpy.arange(tiff.bounds.left, tiff.bounds.right, tiff.res[0])

        isochroneJson = {
            'crs': {'type':'name','properties':{'name': crsEpsg}},
            'type': 'FeatureCollection',
            'features': []
        }    
        
        figure = plt.figure().add_subplot(111)
        
        # Loop over percentile values
        for percentileIndex in range(0, len(percentileValues)):
            print(percentileValues[percentileIndex])
            values =  tiff.read(percentileIndex + 1) # GDAL 1-based indexing
            values[values == 0] = 120
            contourf = figure.contourf(lonRange, latRange, values, levels = isochroneIntervals)
            contours = geojsoncontour.contourf_to_geojson(contourf = contourf, ndigits = 3)
            contourJson = json.loads(contours)
            print(len(contourJson['features']))
            
            # Loop over the travel time cutoff values 
            # (one set of features per element of isochrone intervals)
            for i in range(len(contourJson['features'])):

                feature = contourJson['features'][i]
                cutoff = list(isochroneIntervals)[i+1]

                isochroneFeature = {
                    'type': 'Feature',
                    'properties': {
                        'minutes': cutoff,
                        'percentile': percentileValues[percentileIndex]
                    }
                }

                # The next two steps remove shapes with no coordinates, zero-area artifacts, 
                # and self-intersections
                nextBand = shape({
                    'type': 'MultiPolygon', 
                    'coordinates': list(filter(
                        lambda coords: (len(coords) > 0), 
                        feature['geometry']['coordinates']))
                })

                nextBand = MultiPolygon(list(filter(lambda poly: (poly.area > 0), nextBand))).buffer(0)

                # countourf returns the area between the requested intervals (e.g. a bandreachable in 
                # between 15 and 30 minutes). Union this band with the previous ones to show the cumulative
                # area reachable.
                if i > 0:
                    nextBand = nextBand.union(shape(isochroneJson['features'][i - 1]['geometry']))

                isochroneFeature['geometry'] = json.loads(shapely_geojson.dumps(nextBand))
                isochroneJson['features'].append(isochroneFeature)

            # Write out the various travel time cutoff and percentile values to a single file per-origin.
            with open(originName + '.geojson', 'w') as f:
                json.dump(isochroneJson, f)
            f.close()

## Download accessibility results

**NOTE:** Your profile request JSON file *must* have an ID of spatial dataset defined in `destinationPointSetIds` for this section to work. To find a spatial dataset ID, navigate to your spatial dataset and use the `opportunityDatasetId` defined at the end of the URL. For example:

URL: `https://analysis.conveyal.com/regions/5f3b4199f0b14a34c61a0653/opportunities?opportunityDatasetId=61b751e1f49a7e2152fa209b`
Profile Request JSON entry: `"destinationPointSetIds": ["61b751e1f49a7e2152fa209b"]`

In [ ]:
# Switch the accept header from tiff (used for geotiff above) to json
headers['Accept'] = 'application/json'

# See https://github.com/conveyal/analysis-ui/blob/dev/lib/actions/analysis/parse-times-data.ts
headerEntries = 7
bytesPerElement = 4
timesGridType = 'ACCESSGR'
headerStart = len(timesGridType)
headerEnd = headerStart + bytesPerElement * headerEntries

combinedAccessibility = {}

with open('batch-accessibility.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(['origin', 'percentile', 'cutoff', 'accessibility'])

    # Loop over origins
    for origin in origins:
        originName = origin['name']
        combinedAccessibility[originName] = {}
        profileRequest['fromLat'] = origin['fromLat']
        profileRequest['fromLon'] = origin['fromLon']

        print('Processing ' + originName)


        # Request a single-point analysis from Conveyal
        r = requests.post(url, headers = headers, json = profileRequest, verify = False)

        if r.status_code == 202:
            print('Routing engine starting, try again in a few minutes.')

        elif r.status_code == 403:
            print('Unauthorized access. Your authorization token may be invalid or expired.')

        elif r.status_code != 200:
            print('Error: ' + r.text)

        else:
            # little-endian, unsigned int
            header = unpack('<'+ str(headerEntries) + 'I', r.content[headerStart : headerEnd])
            version = header[0]
            zoom = header[1]
            west = header[2]
            north = header[3]
            width = header[4]
            height = header[5]
            depth = header[6]
            accessibilityResults = json.loads(r.content[headerEnd + bytesPerElement * height * width * depth :])['accessibility']
            for d in range(depth):
                percentile = profileRequest['percentiles'][d]
                for c in range(120):
                    writer.writerow([originName, percentile, c + 1, accessibilityResults[0][d][c]])
csvFile.close()